In [ ]:
!python -V

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install scikit-learn

In [ ]:
!pip install pylance

### Setup env

In [ ]:
import pandas as pd
#import pickle
#import seaborn as sns
#import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

### First exploration

In [ ]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df['duration'].describe()

In [ ]:
df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [ ]:
# Check the first few rows of the DataFrame
df_short = df[(df.duration >= 1) & (df.duration <= 60)]

#categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_short[categorical] = df_short[categorical].astype(str)
df_short['duration'].count()
df_short['duration'].describe()
df_short['duration'].count()/df['duration'].count()

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

In [ ]:
sns.histplot(y_pred, label='prediction')
sns.histplot(y_train, label='actual')

plt.legend()

## Main

Creating the import function and cleaning.

In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

## Importing Data

In [4]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U1')) -> None

In [ ]:
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

Doublecheck sizes

In [ ]:
len(df_train), len(df_val)

In [ ]:

df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:


dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)